In [2]:
import os

root_dir = "wiki-small/"

output_trec_file = "ejemplo.trec"

def enumerate_html_files(directory):
    document_id = 1
    with open(output_trec_file, 'w') as trec_file:
        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith(".html"):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r', encoding='utf-8') as html_file:
                        content = html_file.read()
                        trec_file.write(f"<DOC>\n")
                        trec_file.write(f"<DOCNO>{document_id}</DOCNO>\n")
                        trec_file.write(f"<TEXT>\n")
                        trec_file.write(f"{content}\n")
                        trec_file.write(f"</TEXT>\n")
                        trec_file.write(f"</DOC>\n")
                        # Incrementar el ID del documento
                        document_id += 1

enumerate_html_files(root_dir)



In [3]:
import pyterrier as pt

if not pt.started():
    pt.init()

path_to_collection = "ejemplo.trec"

indexer = pt.TRECCollectionIndexer("./index", verbose=True, overwrite=True)

indexref =indexer.index(path_to_collection)
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
2files [00:17,  8.72s/files]            

Number of documents: 6043
Number of terms: 164940
Number of postings: 1689407
Number of fields: 0
Number of tokens: 3128731
Field names: []
Positions:   false



In [29]:
# ad-hoc retrieval
import numpy as np
br = pt.BatchRetrieve(index, wmodel="TF_IDF") # wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"}
results = br.search("software")
tfidf = np.array(results["docno"]).astype(int)
tfidf.tolist()

[5856,
 3571,
 1450,
 5176,
 5942,
 2499,
 3882,
 1031,
 3767,
 1273,
 3520,
 805,
 955,
 3934,
 2111,
 3482,
 2495,
 1353,
 2727,
 3656,
 1836,
 5022,
 956,
 206,
 899,
 3749,
 4401,
 1970,
 2548,
 4692,
 1746,
 1218,
 1243,
 3648,
 2626,
 1740,
 1049,
 5059,
 1363,
 198,
 713,
 3375,
 1208,
 3883,
 2523,
 5001,
 1179,
 5587,
 2861,
 2829,
 2979,
 61,
 1040,
 134,
 3318,
 5798,
 2978,
 4332,
 5931,
 54,
 141,
 321,
 5491,
 5493,
 35,
 2083,
 4100,
 2153,
 1444,
 5198,
 3517,
 4700,
 3758,
 642,
 5129,
 151,
 4261,
 5020,
 3123,
 197,
 5561,
 4075,
 4810,
 4215,
 325,
 2819,
 3,
 3519,
 4713,
 2767,
 2233,
 2014,
 2331,
 3630,
 2376,
 1106,
 2808,
 22,
 1314,
 3978,
 5196,
 2276,
 2098,
 4440,
 3491,
 3392,
 40,
 2486,
 783,
 2318,
 5728,
 1439,
 880,
 1543]

In [30]:
# ad-hoc retrieval
br = pt.BatchRetrieve(index, wmodel="BM25") # wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"}
results = br.search("software")
results
bm25 = np.array(results["docno"]).astype(int)
bm25.tolist()

[5856,
 3571,
 1450,
 5176,
 5942,
 2499,
 3882,
 1031,
 3767,
 1273,
 3520,
 805,
 955,
 3934,
 2111,
 3482,
 2495,
 1353,
 2727,
 3656,
 1836,
 5022,
 956,
 206,
 899,
 3749,
 4401,
 1970,
 2548,
 4692,
 1746,
 1218,
 1243,
 3648,
 2626,
 1740,
 1049,
 5059,
 1363,
 198,
 713,
 3375,
 1208,
 3883,
 2523,
 5001,
 1179,
 5587,
 2861,
 2829,
 2979,
 61,
 1040,
 134,
 3318,
 5798,
 2978,
 4332,
 5931,
 54,
 141,
 321,
 5491,
 5493,
 35,
 2083,
 4100,
 2153,
 1444,
 5198,
 3517,
 4700,
 3758,
 642,
 5129,
 151,
 4261,
 5020,
 3123,
 197,
 5561,
 4075,
 4810,
 4215,
 325,
 2819,
 3,
 3519,
 4713,
 2767,
 2233,
 2014,
 2331,
 3630,
 2376,
 1106,
 2808,
 22,
 1314,
 3978,
 5196,
 2276,
 2098,
 4440,
 3491,
 3392,
 40,
 2486,
 783,
 2318,
 5728,
 1439,
 880,
 1543]

In [31]:
import pingouin as pg
display(pg.corr(tfidf, bm25, method='spearman'))

,n,r,CI95%,p-val,power
spearman,114,1.0,"[1.0, 1.0]",0.0,1
